해당 노트북은 YouTube [todaycode오늘코드](https://www.youtube.com/channel/UCLR3sD0KB_dWpvcsrLP0aUg) 강의를 기반으로 작성했습니다.

# 1. 서울시에서 공개한 코로나19 발생동향 분석
* https://www.seoul.go.kr/coronaV/coronaStatus.do

* 11월 중순 이후 확진자가 증가함에 따라 '확진자 현황' 데이터를 여러개 페이지(1, 2, 3, ..)로 구분하여 웹사이트에 공개! -> 기존 pd.read_html()로 데이터 불러올 수 없음
* read_html()로는 칼럼명을 불러오고, 데이터는 requests 라이브러리로 읽어오는 방식 적용
* <font color='red'>보통 웹사이트에서 데이터를 크롤링 하여 .csv 파일로 저장한 후, 이를 다시 불러와서 데이터 분석을 진행한다!</font>

In [1]:
# 필요한 라이브러리
import pandas as pd
import numpy as np

In [2]:
# 크롤링으로 가져올 url 주소
url = "https://www.seoul.go.kr/coronaV/coronaStatus.do"
url

'https://www.seoul.go.kr/coronaV/coronaStatus.do'

In [3]:
pd.read_html?

In [4]:
# read_html로 url 웹 사이트에서 별도의 로그인 없이 볼 수 있는 html의 table 태그(<tabel class ...)가 있는 데이터를 모두 불러옴!
table = pd.read_html(url)

In [5]:
# 테이블 개수는 잘 가져옴
len(table)

6

In [6]:
# 첫 번째 table: 구별 누적 확진자수 및 특정 시간 기준 추가 확진자수
table[0] # PC 화면용 테이블

,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,강북구,도봉구,노원구,은평구,서대문구
0,515,373,533,591,590,893,1008,1029,629,745,1037,976,664
1,+3,+3,+6,+4,+6,+14,+5,+8,+7,+3,+4,+1,+6
2,마포구,양천구,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구,기타
3,855,899,1562,787,396,886,1013,1192,972,1165,1364,750,2971
4,+8,0,+4,+4,+2,+4,+3,+5,+2,+6,+4,+6,+10


In [7]:
# 
table[1] # 모바일 화면용 테이블

,종로구,중구,용산구,성동구,광진구,동대문구
0,515,373,533,591,590,893
1,+3,+3,+6,+4,+6,+14
2,중랑구,성북구,강북구,도봉구,노원구,은평구
3,1008,1029,629,745,1037,976
4,+5,+8,+7,+3,+4,+1
5,서대문구,마포구,양천구,강서구,구로구,금천구
6,664,855,899,1562,787,396
7,+6,+8,0,+4,+4,+2
8,영등포구,동작구,관악구,서초구,강남구,송파구
9,886,1013,1192,972,1165,1364


In [8]:
# 11월 중순 확진자 증가로 인해 테이블 데이터를 페이지(1, 2, 3, 4, ...)를 나눠서 공개! -> read_html()로 불러올 수 없다!
df = table[3]
df.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황


## 11월 중순 이후 데이터 공개방식 변경
* [requests 라이브러리 설명](https://en.wikipedia.org/wiki/Requests_(software))

In [9]:
# requests: 작은 브라우저로 웹사이트를 읽어오는 목적!
# -> 웹사이트의 이미지, 텍스트, 표의 HTML 코드만 반환!
import requests 

* 1페이지 테이블에 대한 url을 알고 싶을 때, 테이블 우클릭 -> 검사 -> Network 탭 -> XHR -> 해당 draw에 맞는 get_status_ajax.php 선택 -> Headers -> Response URL 전체 주소 복사!
<img src="img/draw_1_URL.PNG" />

In [10]:
# '1페이지 테이블'의 Request URL을 전체 복사한후, columns, start 기준으로 f-string으로 문자열 연결!
# get_status_ajax.php가 웹사이트 '확진자 현황' 탭 데이터를 가지고 있음!
url = "https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1"
url = f"{url}&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=desc"
url = f"{url}&start=0&length=100&search%5Bvalue%5D=&search%5Bregex%5D=true&_=1612249942210"
url


'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&colum

In [11]:
# url 상태 확인(?): 200이면, 정상적으로 받아왔음을 의미!
response = requests.get(url)
response

<Response [200]>

In [12]:
# response.json(): '1페이지 테이블' 데이터 반환! 
data_json = response.json()
data_json

{'draw': 1,
 'recordsTotal': 14395,
 'recordsFiltered': 14395,
 'data': [["<p class='corona19_no'>24395</p>",
   '78560',
   '2021-01-31',
   '마포구',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24394</p>",
   '78588',
   '2021-02-01',
   '타시도',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24393</p>",
   '78822',
   '2021-02-01',
   '강남구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24392</p>",
   '78826',
   '2021-02-01',
   '관악구',
   '-',
   '광진구 음식점 관련',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24391</p>",
   '78825',
   '2021-02-01',
   '관악구',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24390</p>",
   '78824',
   '2021-02-01',
   '관악구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24389</p>",
   '78798',
   '2021-02-01',
   '영등포구',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24388</p>",
   '78802'

In [13]:
# 1페이지 중, url에 지정한 start=0&length=100만큼의 데이터만 가져왔지만, 웹사이트 1페이지~끝 페이지까지의 모든 데이터 개수도 알 수 있음
data_json["recordsTotal"]

14395

In [14]:
# 그중에 url에 지정한  에 의해 처음부터 100개 데이터만 가져옴! 
pd.DataFrame(data_json["data"])

,0,1,2,3,4,5,6
0,<p class='corona19_no'>24395</p>,78560,2021-01-31,마포구,-,감염경로 조사중,<b class=''></b>
1,<p class='corona19_no'>24394</p>,78588,2021-02-01,타시도,-,감염경로 조사중,<b class=''></b>
2,<p class='corona19_no'>24393</p>,78822,2021-02-01,강남구,-,기타 확진자 접촉,<b class=''></b>
3,<p class='corona19_no'>24392</p>,78826,2021-02-01,관악구,-,광진구 음식점 관련,<b class=''></b>
4,<p class='corona19_no'>24391</p>,78825,2021-02-01,관악구,-,감염경로 조사중,<b class=''></b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>24300</p>,78593,2021-02-01,금천구,-,감염경로 조사중,<b class=''></b>
96,<p class='corona19_no'>24299</p>,78559,2021-02-01,강북구,-,감염경로 조사중,<b class=''></b>
97,<p class='corona19_no'>24298</p>,78575,2021-02-01,동대문구,-,동대문구 소재 고시텔 관련,<b class=''></b>
98,<p class='corona19_no'>24297</p>,78642,2021-02-01,강서구,-,기타 확진자 접촉,<b class=''></b>


In [15]:
# **위 특정 페이지 테이블의 Request URL이 전부 필요하지는 않고, draw, start(데이터 시작 인덱스 -> 1페이지가 10개 데이터씩 있다면, draw=2(2페이지) 데이터 일때는 start=10이어야 함!), length(한 페이지 데이터 길이)만 있어도 데이터를 가져올 수 있음

# draw: 웹사이트 페이지 번호를 누를때마다 1씩 증가!
# => draw=1: 최초 1페이지 테이블을 가져올 수 있음! 
url = "https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1"
url = url + "&order%5B0%5D%5Bdir%5D=desc&start=0&length=100"
url

'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1&order%5B0%5D%5Bdir%5D=desc&start=0&length=100'

In [16]:
response = requests.get(url)
data_json = response.json()

In [17]:
data_json

{'draw': 1,
 'recordsTotal': 14395,
 'recordsFiltered': 14395,
 'data': [["<p class='corona19_no'>24395</p>",
   '78560',
   '2021-01-31',
   '마포구',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24394</p>",
   '78588',
   '2021-02-01',
   '타시도',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24393</p>",
   '78822',
   '2021-02-01',
   '강남구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24392</p>",
   '78826',
   '2021-02-01',
   '관악구',
   '-',
   '광진구 음식점 관련',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24391</p>",
   '78825',
   '2021-02-01',
   '관악구',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24390</p>",
   '78824',
   '2021-02-01',
   '관악구',
   '-',
   '기타 확진자 접촉',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24389</p>",
   '78798',
   '2021-02-01',
   '영등포구',
   '-',
   '감염경로 조사중',
   "<b class=''></b>"],
  ["<p class='corona19_no'>24388</p>",
   '78802'

In [18]:
# records_total: 웹페이지 1~끝페이지 데이터 개수
records_total = data_json["recordsTotal"]
# 끝 페이지 번호! -> 개략적인 계산을 위해 +1을 더해줬다는데.
end_page = round(records_total / 100) + 1 
end_page

145

In [19]:
data = data_json["data"]
data

[["<p class='corona19_no'>24395</p>",
  '78560',
  '2021-01-31',
  '마포구',
  '-',
  '감염경로 조사중',
  "<b class=''></b>"],
 ["<p class='corona19_no'>24394</p>",
  '78588',
  '2021-02-01',
  '타시도',
  '-',
  '감염경로 조사중',
  "<b class=''></b>"],
 ["<p class='corona19_no'>24393</p>",
  '78822',
  '2021-02-01',
  '강남구',
  '-',
  '기타 확진자 접촉',
  "<b class=''></b>"],
 ["<p class='corona19_no'>24392</p>",
  '78826',
  '2021-02-01',
  '관악구',
  '-',
  '광진구 음식점 관련',
  "<b class=''></b>"],
 ["<p class='corona19_no'>24391</p>",
  '78825',
  '2021-02-01',
  '관악구',
  '-',
  '감염경로 조사중',
  "<b class=''></b>"],
 ["<p class='corona19_no'>24390</p>",
  '78824',
  '2021-02-01',
  '관악구',
  '-',
  '기타 확진자 접촉',
  "<b class=''></b>"],
 ["<p class='corona19_no'>24389</p>",
  '78798',
  '2021-02-01',
  '영등포구',
  '-',
  '감염경로 조사중',
  "<b class=''></b>"],
 ["<p class='corona19_no'>24388</p>",
  '78802',
  '2021-02-01',
  '노원구',
  '-',
  '감염경로 조사중',
  "<b class=''></b>"],
 ["<p class='corona19_no'>24387</p>",
  '78814',
  

In [20]:
# 0 ~ 6까지 칼럼명이 없음
pd.DataFrame(data)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>24395</p>,78560,2021-01-31,마포구,-,감염경로 조사중,<b class=''></b>
1,<p class='corona19_no'>24394</p>,78588,2021-02-01,타시도,-,감염경로 조사중,<b class=''></b>
2,<p class='corona19_no'>24393</p>,78822,2021-02-01,강남구,-,기타 확진자 접촉,<b class=''></b>
3,<p class='corona19_no'>24392</p>,78826,2021-02-01,관악구,-,광진구 음식점 관련,<b class=''></b>
4,<p class='corona19_no'>24391</p>,78825,2021-02-01,관악구,-,감염경로 조사중,<b class=''></b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>24300</p>,78593,2021-02-01,금천구,-,감염경로 조사중,<b class=''></b>
96,<p class='corona19_no'>24299</p>,78559,2021-02-01,강북구,-,감염경로 조사중,<b class=''></b>
97,<p class='corona19_no'>24298</p>,78575,2021-02-01,동대문구,-,동대문구 소재 고시텔 관련,<b class=''></b>
98,<p class='corona19_no'>24297</p>,78642,2021-02-01,강서구,-,기타 확진자 접촉,<b class=''></b>


## 위 기능과 동일하게 1페이지만 가져오는 함수 정의

In [21]:
# 1) 확진자 현황 (10001번 ~ 현재)
def get_seoul_covid19_10001_current(page_no):
    
    start_no = (page_no - 1) * 100
    
    url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw={page_no}"
    url = f"{url}&order%5B0%5D%5Bdir%5D=desc&start={start_no}&length=100"
    
    response = requests.get(url)
    data_json = response.json()
    
    
    return data_json

In [22]:
# test add
# 2) 확진자 현황 (1번 ~ 10000번)
def get_seoul_covid19_1_10000(page_no):
    
    # 확진자 현황 (1번 ~ 10000번)의 최초 page_no는 3으로 해야 함!
    start_no = (page_no - 3) * 100
    
    url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax_pre.php?draw={page_no}"
    url = f"{url}&order%5B0%5D%5Bdir%5D=desc&start={start_no}&length=100"
    
        
    response = requests.get(url)
    data_json = response.json()
    
    
    return data_json

## 전체 데이터 가져오기
tqdm 사용!
* 오래 걸리는 작업의 진행상태 표시


In [23]:
# time.sleep으로 시차를 두기 위함
import time

# tqdm: 진행 상태 표시를 위함 
from tqdm import trange

* 1) 일단 1~3페이지만 데이터가 제대로 로드되는지 확인 
    * 처음부터 전체 페이지를 수집하면, 중간에 오류가 나도 찾기 어려움
    * 일부만 우선 가져와 보고, 잘 동작하면 나머지 전체 가져오기!

In [25]:

page_list = []

all_page = 3
for page_no in trange(all_page+1): # for ~ in range()처럼 all_page + 1을 해준다! 
    one_page_data_json = get_seoul_covid19_10001_current(page_no)
    one_page_df = pd.DataFrame(one_page_data_json["data"])
    page_list.append(one_page_df)
    
    # 서버에 한 번에 너무 많은 요청을 보내면 부담이 되므로.. 
    # 0.5초씩 쉬었다가 크롤링 수행
    time.sleep(0.5)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.01it/s]


In [26]:
# 3페이지 데이터까지 잘 가져옴!
pd.concat(page_list)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>24395</p>,78560,2021-01-31,마포구,-,감염경로 조사중,<b class=''></b>
1,<p class='corona19_no'>24394</p>,78588,2021-02-01,타시도,-,감염경로 조사중,<b class=''></b>
2,<p class='corona19_no'>24393</p>,78822,2021-02-01,강남구,-,기타 확진자 접촉,<b class=''></b>
3,<p class='corona19_no'>24392</p>,78826,2021-02-01,관악구,-,광진구 음식점 관련,<b class=''></b>
4,<p class='corona19_no'>24391</p>,78825,2021-02-01,관악구,-,감염경로 조사중,<b class=''></b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>24100</p>,78014,2021-01-30,강동구,-,감염경로 조사중,<b class=''></b>
96,<p class='corona19_no'>24099</p>,78043,2021-01-30,노원구,-,강남구 소재 직장3 관련,<b class=''></b>
97,<p class='corona19_no'>24098</p>,77976,2021-01-30,구로구,-,감염경로 조사중,<b class=''></b>
98,<p class='corona19_no'>24097</p>,77975,2021-01-30,성동구,-,성동구 소재 병원 관련(\\\'21.1월),<b class=''></b>


* 2) 전체 페이지 데이터 로드

In [27]:
# 1) 확진자 현황 (10001번 ~ 현재)
def get_multi_page_list_10001_current(start_page, end_page=80):
    
    page_list = []
    for page_no in trange(start_page, end_page + 1):
        one_page_data_json = get_seoul_covid19_10001_current(page_no)
        
        if len(one_page_data_json["data"]) > 0:
            one_page_df = pd.DataFrame(one_page_data_json["data"])
            page_list.append(one_page_df)
            time.sleep(0.5)
        else:
            # 수집된 값이 없으면, page_list를 곧바로 반환
            return page_list
    
    return page_list

In [34]:
# 2) 확진자 현황 (1번 ~ 10000번)
def get_multi_page_list_1_10000(start_page, end_page=80):
    
    page_list = []
    for page_no in trange(start_page, end_page + 1):
        #print(page_no)
        one_page_data_json = get_seoul_covid19_1_10000(page_no)
        
        if len(one_page_data_json["data"]) > 0:
            one_page_df = pd.DataFrame(one_page_data_json["data"])
            page_list.append(one_page_df)
            time.sleep(0.5)
        else:
            # 수집된 값이 없으면, page_list를 곧바로 반환
            return page_list
    
    return page_list

In [30]:
# 1) 확진자 현황 (10001번 ~ 현재): 2021.02.02 기준 1페이지 ~ 144페이지
start_page = 1
page_list = get_multi_page_list_10001_current(start_page, end_page) #  1페이지 ~ end_page(144) 페이지까지의 데이터 크롤링
# 데이터가 많아서 1페이지 데이터만 확인
#page_list[:1]
df_all_10001_current = pd.concat(page_list)

#df_all_10001_current.shape
#df_all_10001_current.head(1)

# read_html()로 읽어온 table[3]의 칼럼명을 리스트로 만들어서 df_all에 적용
df_all_10001_current.columns = table[3].columns.tolist()
df_all_10001_current.head()

 99%|███████████████████████████████████████████████████████████████████████████████▍| 144/145 [02:14<00:00,  1.07it/s]


,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,<p class='corona19_no'>24395</p>,78560,2021-01-31,마포구,-,감염경로 조사중,<b class=''></b>
1,<p class='corona19_no'>24394</p>,78588,2021-02-01,타시도,-,감염경로 조사중,<b class=''></b>
2,<p class='corona19_no'>24393</p>,78822,2021-02-01,강남구,-,기타 확진자 접촉,<b class=''></b>
3,<p class='corona19_no'>24392</p>,78826,2021-02-01,관악구,-,광진구 음식점 관련,<b class=''></b>
4,<p class='corona19_no'>24391</p>,78825,2021-02-01,관악구,-,감염경로 조사중,<b class=''></b>


In [37]:
# 2) 확진자 현황 (1번 ~ 10000번): 2021.02.02 기준 1페이지 ~ 100페이지
start_page = 3
end_page = 103
page_list = get_multi_page_list_1_10000(start_page, end_page) #  1페이지 ~ end_page(100) 페이지까지의 데이터 크롤링
# 데이터가 많아서 1페이지 데이터만 확인
#page_list[:1]


df_all_1_10000 = pd.concat(page_list)

#df_all_1_10000.shape
#df_all_1_10000.head(1)

# read_html()로 읽어온 df_all_10001_current의 칼럼명을 리스트로 만들어서 df_all에 적용
df_all_1_10000.columns = df_all_10001_current.columns.tolist()
df_all_1_10000.head()


 99%|███████████████████████████████████████████████████████████████████████████████▏| 100/101 [01:29<00:00,  1.12it/s]


,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,<p class='corona19_no'>10000</p>,37131,2020-12-05,금천구,-,구로구 소재 보험회사 관련,<b class='status1'>퇴원</b>
1,<p class='corona19_no'>9999</p>,37141,2020-12-05,금천구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
2,<p class='corona19_no'>9998</p>,37541,2020-12-05,송파구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
3,<p class='corona19_no'>9997</p>,37542,2020-12-05,송파구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
4,<p class='corona19_no'>9996</p>,37087,2020-12-05,서초구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>


In [45]:
# 최종 데이터 결합! -> 최초 서울시 코로나 확진자 발생 ~ 2021년2월2일 기준까지의 확진자 데이터!
df_all = pd.concat([df_all_10001_current, df_all_1_10000])
df_all

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,<p class='corona19_no'>24395</p>,78560,2021-01-31,마포구,-,감염경로 조사중,<b class=''></b>
1,<p class='corona19_no'>24394</p>,78588,2021-02-01,타시도,-,감염경로 조사중,<b class=''></b>
2,<p class='corona19_no'>24393</p>,78822,2021-02-01,강남구,-,기타 확진자 접촉,<b class=''></b>
3,<p class='corona19_no'>24392</p>,78826,2021-02-01,관악구,-,광진구 음식점 관련,<b class=''></b>
4,<p class='corona19_no'>24391</p>,78825,2021-02-01,관악구,-,감염경로 조사중,<b class=''></b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>5</p>,9,2020-01-31,성북구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
96,<p class='corona19_no'>4</p>,7,2020-01-30,마포구,중국,해외유입,<b class='status1'>퇴원</b>
97,<p class='corona19_no'>3</p>,6,2020-01-30,종로구,-,종로구 집단발병,<b class='status1'>퇴원</b>
98,<p class='corona19_no'>2</p>,5,2020-01-30,중랑구,중국,해외유입,<b class='status1'>퇴원</b>


* 가장 첫 번째 row의 확진일이 2021-02-01이 들어가야는데 웹사이트 데이터의 오타로 보임!
* 연번, 퇴원현황 칼럼 데이터 전처리가 필요해보임

## 데이터 전처리 (연번, 퇴원현황 칼럼)
* re 라이브러리로 정규 표현식을 사용하여 필요한 글자만 추출

In [41]:
import re

* 1) 연번 데이터

In [42]:
# 테스트

num_string = "<p class='corona19_no'>23797</p>"
# num_string: "<p class='_no'>23797</p>"로 변환 
# -> 정규표현식으로 숫자를 제외한 문자를 없애려고 하는데, 23797 이외에 필요없는 숫자 '19'가 있다!
# -> 그래서 먼저 .replace()를 사용해서 '19'를 포함한 'corona19'를 공백으로 바꿔준다!
num_string = num_string.replace("corona19", "") 

# re.sub('패턴', '바꿀 문자열', '문자열', 바꿀횟수)
# -> num_string에서 정규표현식으로 추출한 문자열을 ""로 바꾼다! -> 즉, 정규표현식으로 추출한 문자를 제외할 수 있음!
# num: 23797로 저장
num = re.sub("[^0-9]", "", num_string)  
num

'23797'

In [47]:
def extract_number(num_string):
    # 실수 방지 차원에서의 num_string 타입 검사?
    if type(num_string) == str:
        num_string = num_string.replace("corona19", "")
        num = re.sub("[^0-9]", "", num_string)
        num = int(num)
        return num
    else:
        return num_string

In [48]:
# "연번" 데이터를 1개씩 extract_number() 함수에 입력
df_all["연번"] = df_all["연번"].map(extract_number)
# 연변 데이터 숫자만 추출 완료!
df_all

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,24395,78560,2021-01-31,마포구,-,감염경로 조사중,<b class=''></b>
1,24394,78588,2021-02-01,타시도,-,감염경로 조사중,<b class=''></b>
2,24393,78822,2021-02-01,강남구,-,기타 확진자 접촉,<b class=''></b>
3,24392,78826,2021-02-01,관악구,-,광진구 음식점 관련,<b class=''></b>
4,24391,78825,2021-02-01,관악구,-,감염경로 조사중,<b class=''></b>
...,...,...,...,...,...,...,...
95,5,9,2020-01-31,성북구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
96,4,7,2020-01-30,마포구,중국,해외유입,<b class='status1'>퇴원</b>
97,3,6,2020-01-30,종로구,-,종로구 집단발병,<b class='status1'>퇴원</b>
98,2,5,2020-01-30,중랑구,중국,해외유입,<b class='status1'>퇴원</b>


* 2) 퇴원현황 데이터

In [49]:
df_all["퇴원현황"].value_counts()

<b class='status1'>퇴원</b>    20451
<b class='status1'></b>       2898
<b class=''></b>               649
<b class='status2'>사망</b>      326
<b class='status2'></b>         71
Name: 퇴원현황, dtype: int64

In [50]:
# 테스트 

string = "<b class='status1'>퇴원</b> "
# 한글을 제외한 모든 문자를 "" 공백으로 변환! 
re.sub("[^가-힣]", "", string)

'퇴원'

In [51]:
def extract_hangeul(origin_text):
    subtract_text = re.sub("[^가-힣]", "", origin_text)
    return subtract_text

In [52]:
df_all["퇴원현황"] = df_all["퇴원현황"].map(extract_hangeul)
# 퇴원현황이 빈 값(퇴원도 사망도 아닌 사람)은 Nan으로 채우기!
df_all.loc[df_all["퇴원현황"].isin([""]), "퇴원현황"] = np.nan
# 퇴원현황 전처리 완료!
df_all

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,24395,78560,2021-01-31,마포구,-,감염경로 조사중,NaN
1,24394,78588,2021-02-01,타시도,-,감염경로 조사중,NaN
2,24393,78822,2021-02-01,강남구,-,기타 확진자 접촉,NaN
3,24392,78826,2021-02-01,관악구,-,광진구 음식점 관련,NaN
4,24391,78825,2021-02-01,관악구,-,감염경로 조사중,NaN
...,...,...,...,...,...,...,...
95,5,9,2020-01-31,성북구,-,기타 확진자 접촉,퇴원
96,4,7,2020-01-30,마포구,중국,해외유입,퇴원
97,3,6,2020-01-30,종로구,-,종로구 집단발병,퇴원
98,2,5,2020-01-30,중랑구,중국,해외유입,퇴원


In [53]:
df_all["퇴원현황"].unique()

array([nan, '사망', '퇴원'], dtype=object)

* 가장 최근 확진일 날짜 기준으로 .csv 파일로 저장

In [54]:
'''
(추가 설정) 
가장 최근 확진일이 웹사이트에 잘못 기입된 것으로 보아 2020-02-01로 변경 -> 웹사이트에서 변경되면, 해당 내용 삭제!
'''
df_all.iloc[0, 2] = '2021-02-01'

In [55]:
# 가장 최근 확진일 날짜
last_date = df_all.iloc[0]["확진일"]
last_date

'2021-02-01'

In [57]:
df_all.head(1)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,24395,78560,2021-02-01,마포구,-,감염경로 조사중,NaN


In [56]:
# 확진일 날짜의 연-월-일 구분자를 '_'로 변경
date = last_date.replace("-", "_")
date

'2021_02_01'

In [58]:
file_name = f"seoul-covid19-{date}.csv"
file_name

'seoul-covid19-2021_02_01.csv'

In [59]:
# encoding 인자는 기본적으로 utf-8로 저장! 
# -> 주피터 노트북에서 한글을 보려면, utf-8로 저장! => 즉, encoding 인자를 명시하지 않아도 됨
# -> 단, 엑셀 파일에서 한글을 보려면, 'cp949'로 저장!
# --> 보통은 엑셀 파일을 직접 열어보지 않으므로, encoding 인자를 명시하지 않고 utf-8로 저장함!
df_all.to_csv(f"data/{file_name}", index=False)

In [60]:
# 저장한 .csv 확인
pd.read_csv(f"data/{file_name}")

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,24395,78560,2021-02-01,마포구,-,감염경로 조사중,NaN
1,24394,78588,2021-02-01,타시도,-,감염경로 조사중,NaN
2,24393,78822,2021-02-01,강남구,-,기타 확진자 접촉,NaN
3,24392,78826,2021-02-01,관악구,-,광진구 음식점 관련,NaN
4,24391,78825,2021-02-01,관악구,-,감염경로 조사중,NaN
...,...,...,...,...,...,...,...
24390,5,9,2020-01-31,성북구,-,기타 확진자 접촉,퇴원
24391,4,7,2020-01-30,마포구,중국,해외유입,퇴원
24392,3,6,2020-01-30,종로구,-,종로구 집단발병,퇴원
24393,2,5,2020-01-30,중랑구,중국,해외유입,퇴원
